<a href="https://colab.research.google.com/github/oshan35/Epileptic-Seizure-Prediction-with-EEG/blob/main/Time_Clip_Segments_Non_Sezisure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mne
!pip install numpy
!pip install matplotlib
!pip install pandas
!pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 15.0 MB/s eta 0:00:00


In [12]:
import logging

def create_log():
  for handler in logging.root.handlers[:]:
      logging.root.removeHandler(handler)

  logging.basicConfig(filename='preprocess-pipeline.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
create_log()

In [27]:
import re

class EEGData:
    def __init__(self, file_name, file_start_time, file_end_time, num_seizures, seizure_times=[]):
        self.file_name = file_name
        self.file_start_time = file_start_time
        self.file_end_time = file_end_time
        self.num_seizures = num_seizures
        self.seizure_times = seizure_times


    def __repr__(self):
        return (f"EEGData(File Name: {self.file_name}, "
                f"File Start Time: {self.file_start_time}, "
                f"File End Time: {self.file_end_time}, "
                f"Number of Seizures: {self.num_seizures}, "
                f"Number of Seizures: {self.seizure_times})")

In [ ]:
# def parse_eeg_data(file_path):
#     eeg_data_objects = []

#     with open(file_path, 'r') as file:
#         lines = file.readlines()

#     current_file_data = {}
#     for line in lines:
#         if line.startswith('File Name:'):
#             if current_file_data:
#                 # Create EEGData object from the current_file_data
#                 eeg_data = EEGData(**current_file_data)
#                 eeg_data_objects.append(eeg_data)

#                 current_file_data = {}
#             current_file_data['file_name'] = line.split(': ')[1].strip()
#         elif line.startswith('File Start Time:'):
#             current_file_data['file_start_time'] = line.split(': ')[1].strip()
#         elif line.startswith('File End Time:'):
#             current_file_data['file_end_time'] = line.split(': ')[1].strip()
#         elif line.startswith('Number of Seizures in File:'):
#             current_file_data['num_seizures'] = int(line.split(': ')[1].strip())
#         elif line.startswith('Seizure Start Time:'):
#             current_file_data['seizure_start_time'] = line.split(': ')[1].strip()
#         elif line.startswith('Seizure End Time:'):
#             current_file_data['seizure_end_time'] = line.split(': ')[1].strip()
#             # In case the last file also has seizure information, we need to ensure it gets added
#             if 'file_name' in current_file_data:
#                 eeg_data = EEGData(**current_file_data)
#                 eeg_data_objects.append(eeg_data)
#                 current_file_data = {}

#     return eeg_data_objects


In [28]:
def parse_eeg_data(file_path):
    eeg_data_objects = []

    with open(file_path, 'r') as file:
        lines = file.readlines()
    current_file_data = {}
    num_seizures = 0
    line_index = 0
    while line_index<len(lines):
      line = lines[line_index]
      if line.startswith('File Name:'):
          if current_file_data:
              # Create EEGData object from the current_file_data
              eeg_data = EEGData(**current_file_data)
              eeg_data_objects.append(eeg_data)

              current_file_data = {}
          current_file_data['file_name'] = line.split(': ')[1].strip()
      elif line.startswith('File Start Time:'):
          current_file_data['file_start_time'] = line.split(': ')[1].strip()
      elif line.startswith('File End Time:'):
          current_file_data['file_end_time'] = line.split(': ')[1].strip()
      elif line.startswith('Number of Seizures in File:'):
          current_file_data['num_seizures'] = int(line.split(': ')[1].strip())
          num_seizures = current_file_data['num_seizures']

          seizure_times = []

          if num_seizures > 1:

            for x in range(num_seizures):
              line_index += 1
              start = int(lines[line_index].split(": ")[1].strip().split(" ")[0].strip())
              line_index += 1
              end = int(lines[line_index].split(": ")[1].strip().split(" ")[0].strip())
              seizure = (start,end)
              seizure_times.append(seizure)

          elif num_seizures  == 1:
            line_index += 1
            start = int(lines[line_index].split(": ")[1].split(" ")[0].strip())
            line_index += 1
            end = int(lines[line_index].split(": ")[1].split(" ")[0].strip())
            seizure_times.append((start,end))

          current_file_data['seizure_times'] = seizure_times

          if 'file_name' in current_file_data:
              eeg_data = EEGData(**current_file_data)
              eeg_data_objects.append(eeg_data)
              current_file_data = {}
      line_index += 1
    return eeg_data_objects


In [ ]:
patient_id = "chb12"
arr = parse_eeg_data(f'/content/drive/MyDrive/EEG-Projects/CHB-MIT/{patient_id}/{patient_id}-summary.txt')
for item in arr:
  print(item)

EEGData(File Name: chb12_06.edf, File Start Time: 22:44:34, File End Time: 23:44:40, Number of Seizures: 2, Number of Seizures: [(1665, 1726), (3415, 3447)]
EEGData(File Name: chb12_08.edf, File Start Time: 00:46:38, File End Time: 1:46:38, Number of Seizures: 4, Number of Seizures: [(1426, 1439), (1591, 1614), (1957, 1977), (2798, 2824)]
EEGData(File Name: chb12_09.edf, File Start Time: 01:46:45, File End Time: 2:46:57, Number of Seizures: 2, Number of Seizures: [(3082, 3114), (3503, 3535)]
EEGData(File Name: chb12_10.edf, File Start Time: 02:47:45, File End Time: 3:47:56, Number of Seizures: 2, Number of Seizures: [(593, 625), (811, 856)]
EEGData(File Name: chb12_11.edf, File Start Time: 03:48:28, File End Time: 4:28:59, Number of Seizures: 1, Number of Seizures: [(1085, 1122)]
EEGData(File Name: chb12_19.edf, File Start Time: 08:57:21, File End Time: 9:57:21, Number of Seizures: 0, Number of Seizures: []
EEGData(File Name: chb12_20.edf, File Start Time: 09:57:27, File End Time: 10:5

In [15]:
import glob
import mne
import gc
from tqdm import tqdm
import numpy as np

def load_data_from_file(patient_id,file_name):
  file = f"/content/drive/MyDrive/EEG-Projects/CHB-MIT/{patient_id}/{file_name}"
  data = mne.io.read_raw_edf(file)
  logging.info(f"loaded EDF file at {file_name} with patient id : {patient_id}")
  return data

In [16]:
import random

def clean_channel_data(data_file):
  data = data_file.get_data()
  logging.info(f"data shape before cleaning: {data.shape}")

  modified_data = []
  channel_num, data_num = data.shape
  if channel_num > 23:
    additional_channel = True
    for channel_index in range(len(data)):
      element = data[channel_index][random.randint(0,data_num-1)]

      if element != -1e-06:
        modified_data.append(data[channel_index])
      else:
        logging.info(f"removed channel index {channel_index}")

    if len(modified_data) > 23:
      modified_data.pop(-1)
      logging.info(f"removed ECG channel")

    new_data = np.array(modified_data)
    logging.info(f"data shape changed from {data.shape} => {new_data.shape}")
    del modified_data,data
    gc.collect()
    return new_data

  logging.info("data is clean")
  return data


In [33]:
def create_non_sezizer_segments(patient_id,eeg_data_objects, segment_length = 7200, time_gap =900):
  count = 0
  for i in range(len(eeg_data_objects)):
    if (int(eeg_data_objects[i].num_seizures) == 0) and i-4 >= 0 and i+3 < len(eeg_data_objects):
      logging.info(f"[+] non-seizure: {eeg_data_objects[i]}")
      flag = True
      index = i-1
      for j in range(4):
        if eeg_data_objects[index].num_seizures != 0:
          flag = False
        index-=1
      index = i
      for k in range(3):
        if eeg_data_objects[index].num_seizures != 0:
          flag = False
        index+=1

      if flag:
        logging.info(f"{eeg_data_objects[i]}, {eeg_data_objects[i-1]}")
        # print(eeg_data_objects[i],eeg_data_objects[i-1])
        obj1 = clean_channel_data(load_data_from_file(patient_id,eeg_data_objects[i].file_name))

        obj2 = clean_channel_data(load_data_from_file(patient_id,eeg_data_objects[i-1].file_name))

        f1_no = eeg_data_objects[i].file_name.split('_')[1].split('.')[0]
        f2_no = eeg_data_objects[i-1].file_name.split('_')[1].split('.')[0]

        conect_data = np.concatenate((obj1, obj2), axis=1)[:segment_length*256]
        logging.info(f"conect data shape non-seizure: {conect_data.shape}")
        logging.info(f"time range non-sezuer: {segment_length/3600}")

        #np.save(f"/content/drive/MyDrive/EEG-Projects/CHB-MIT-2hour-Segments-Non-Seizure/{patient_id}-{f1_no}-{f2_no}.npy",conect_data)

        del obj1
        del obj2
        del conect_data
        gc.collect()
    elif (int(eeg_data_objects[i].num_seizures) > 0) and (i-2 >= 0):
      logging.info(f"[+] seizure: {eeg_data_objects[i]}")
      flag = True


      obj1 = clean_channel_data(load_data_from_file(patient_id,eeg_data_objects[i].file_name))
      obj2 = clean_channel_data(load_data_from_file(patient_id,eeg_data_objects[i-1].file_name))
      obj3 = clean_channel_data(load_data_from_file(patient_id,eeg_data_objects[i-2].file_name))
      time_correction = len(obj2[0])/256 + len(obj3[0])/256
      shape_flag = False

      if obj1.shape[0] == obj2.shape[0] == obj3.shape[0]:

        shape_flag = True

      obj4 = None
      if i-3 >= 0:
        obj4 = clean_channel_data(load_data_from_file(patient_id,eeg_data_objects[i-3].file_name))
        if obj1.shape[0] != obj4.shape[0] and shape_flag == True:
          shape_flag = False

        time_correction += len(obj4[0])/256

      logging.info(f"data shapes: {obj1.shape} {obj2.shape} {obj3.shape}")

      if shape_flag == False:
        logging.info("Shapes doesn't match!")
        continue

      logging.info(f"time correction: {time_correction}")

      timeline = []
      if np.all(obj4 != None):
        timeline = np.concatenate((obj4,obj3,obj2,obj1),axis=1)
      else:
        timeline = np.concatenate((obj3,obj2,obj1),axis=1)

      logging.info(timeline.shape)

      s_count = 0
      for s in eeg_data_objects[i].seizure_times:
        seizure_data = []

        start_time = s[0] + time_correction
        end_time = s[1] + time_correction
        logging.info(f"capture margin: {start_time} - {end_time}")

        end_margin = start_time - time_gap
        start_margin = end_margin - segment_length

        start_data_index = int(start_margin * 256)
        end_data_index = int(end_margin * 256)
        n_channels = timeline.shape[0]
        for x in range(n_channels):

          channel_data = timeline[x][start_data_index:end_data_index]
          seizure_data.append(channel_data)
        np_seizure = np.array(seizure_data,dtype=np.float32)
        logging.info(f"pre-ictal data shape: {np_seizure.shape}")
        s_count+=1
      logging.info(f"time range sezuer: {segment_length/3600}")
      logging.info(f"number of sezuers: {s_count}")

      logging.info(f"data length: {len(seizure_data)}")
      del obj1,obj2,obj3,obj4,seizure_data
      gc.collect()
    count+=1

In [34]:
def load_two_hour_segments(patient_id):
  eeg_data_objects = parse_eeg_data(f'/content/drive/MyDrive/EEG-Projects/CHB-MIT/{patient_id}/{patient_id}-summary.txt')
  create_non_sezizer_segments(patient_id,eeg_data_objects)
  del eeg_data_objects
  gc.collect

In [35]:
gc.collect()

0

In [36]:
!rm "preprocess-pipeline.log"
create_log()

In [32]:
patient_id = "chb13"
load_two_hour_segments(patient_id)

EEGData(File Name: chb13_06.edf, File Start Time: 20:43:41, File End Time: 21:43:41, Number of Seizures: 0, Number of Seizures: []) EEGData(File Name: chb13_05.edf, File Start Time: 19:43:34, File End Time: 20:43:34, Number of Seizures: 0, Number of Seizures: [])
Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb13/chb13_06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-15-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'-'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)
<ipython-input-15-e796011988ef>:9: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3
  data = mne.io.read_raw_edf(file)


Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb13/chb13_05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-15-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'-'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)
<ipython-input-15-e796011988ef>:9: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3
  data = mne.io.read_raw_edf(file)


EEGData(File Name: chb13_07.edf, File Start Time: 21:43:47, File End Time: 22:43:47, Number of Seizures: 0, Number of Seizures: []) EEGData(File Name: chb13_06.edf, File Start Time: 20:43:41, File End Time: 21:43:41, Number of Seizures: 0, Number of Seizures: [])
Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb13/chb13_07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

<ipython-input-15-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8', '-'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)
<ipython-input-15-e796011988ef>:9: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  data = mne.io.read_raw_edf(file)


Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb13/chb13_59.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-15-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8', '-'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)
<ipython-input-15-e796011988ef>:9: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  data = mne.io.read_raw_edf(file)


Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb13/chb13_58.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-15-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8', '-'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)
<ipython-input-15-e796011988ef>:9: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  data = mne.io.read_raw_edf(file)


In [37]:
for i in range(23):
  patient_id = f"chb{i+1:02d}"
  load_two_hour_segments(patient_id)

Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb01/chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-15-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)


Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb01/chb01_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-15-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)


Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb01/chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

<ipython-input-15-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)


Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb23/chb23_07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-15-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)


Extracting EDF parameters from /content/drive/MyDrive/EEG-Projects/CHB-MIT/chb23/chb23_06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-15-e796011988ef>:9: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(file)


##Feature extraction
##Wavelet Coefficents

In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis, entropy
from scipy.signal import welch
from pywt import wavedec

def compute_energy(coefficients):
    if isinstance(coefficients, np.ndarray):
        return np.sum(np.square(np.abs(coefficients))) / len(coefficients)
    elif np.isscalar(coefficients):
        return np.square(np.abs(coefficients))
    else:
        raise ValueError("Unsupported type for coefficients")

def compute_total_energy(approximation_coefficients, detail_coefficients):
    total_energy = 0
    total_energy += compute_energy(approximation_coefficients)
    for detail_coefficient in detail_coefficients:
        total_energy += compute_energy(detail_coefficient)
    return total_energy

def calculate_D_Energy(detail_coefficients):
    total_energy = 0
    for detail_coefficient in detail_coefficients:
        total_energy += compute_energy(detail_coefficient)
    return total_energy


def compute_mean(coefficients):
    return np.mean(coefficients)

def compute_std(coefficients):
    return np.std(coefficients)

def calculate_D_mean(coeffs):
    valid_indices = [i for i in range(1, min(6, len(coeffs)))]
    return np.mean([np.mean(coeffs[i]) for i in valid_indices])


def calculate_A_mean(coeffs):
    return compute_mean(coeffs[0])

def calculate_D_std(coeffs):
    return np.mean([compute_std(coeffs[i]) for i in range(min(6, len(coeffs)))])

def calculate_A_std(coeffs):
    return compute_std(coeffs[0])


def wavelet_feature_extraction(data, type_wav, sampling_frequency, nperseg):
    coefficients = wavedec(data, type_wav, level=5)

    total_energy = compute_total_energy(coefficients[0], coefficients[1:])
    cD_Energy=calculate_D_Energy(coefficients[1:])
    cA_Energy=compute_energy(coefficients[0])
    cD_mean = calculate_D_mean(coefficients[1:])
    cA_mean = calculate_A_mean(coefficients[0])
    cD_std = calculate_D_std(coefficients[1:])
    cA_std = calculate_A_std(coefficients[0])

    return [
        total_energy,
        cD_Energy,
        cA_Energy,
        cD_mean,
        cA_mean,
        cD_std,
        cA_std,
    ]

##Frequency Domain coefficients

In [ ]:
import scipy.signal
import numpy as np

def get_median_frequency(psd):
    median_frequency = np.median(psd)

    return median_frequency

def get_edge_frequency(psd):
    edge_frequency = np.where(psd >= psd.max() / 2)[0][0]

    return edge_frequency

def compute_power_spectral_density(data, sampling_frequency, nperseg=256):
    _, psd = scipy.signal.welch(data, fs=sampling_frequency, nperseg=nperseg)
    return psd

def butter_bandpass(lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = scipy.signal.butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = scipy.signal.lfilter(b, a, data)
    return y

def compute_band_power(psd_result, freq_band_indices, fs, nperseg):
    freq_band_power = np.sum(psd_result[freq_band_indices]) * fs / nperseg
    return freq_band_power

def compute_spectral_entropy(psd):
    normalized_psd = psd / np.sum(psd)  # Normalize to obtain probabilities
    spectral_entropy = -np.sum(normalized_psd * np.log2(normalized_psd))
    return spectral_entropy

def extract_frequency_domain_features(signal, sampling_frequency, nperseg=256):
    # Apply Butterworth bandpass filters
    delta_band_signal = butter_bandpass_filter(signal, 0.5, 4, sampling_frequency)
    theta_band_signal = butter_bandpass_filter(signal, 4, 8, sampling_frequency)
    alpha_band_signal = butter_bandpass_filter(signal, 8, 13, sampling_frequency)
    beta_band_signal = butter_bandpass_filter(signal, 13, 30, sampling_frequency)
    gamma_band_signal = butter_bandpass_filter(signal, 30, 40, sampling_frequency)

    # Compute Power Spectral Density for each band
    delta_psd = compute_power_spectral_density(delta_band_signal, sampling_frequency, nperseg=nperseg)
    theta_psd = compute_power_spectral_density(theta_band_signal, sampling_frequency, nperseg=nperseg)
    alpha_psd = compute_power_spectral_density(alpha_band_signal, sampling_frequency, nperseg=nperseg)
    beta_psd = compute_power_spectral_density(beta_band_signal, sampling_frequency, nperseg=nperseg)
    gamma_psd = compute_power_spectral_density(gamma_band_signal, sampling_frequency, nperseg=nperseg)

    # Compute Band Power for each frequency band
    freq_band_indices = [range(int(nperseg * band[0] / sampling_frequency), int(nperseg * band[1] / sampling_frequency)) for band in [(0.5, 4), (4, 8), (8, 13), (13, 30), (30, 40)]]

    delta_band_power = compute_band_power(delta_psd, freq_band_indices[0], sampling_frequency, nperseg)
    theta_band_power = compute_band_power(theta_psd, freq_band_indices[1], sampling_frequency, nperseg)
    alpha_band_power = compute_band_power(alpha_psd, freq_band_indices[2], sampling_frequency, nperseg)
    beta_band_power = compute_band_power(beta_psd, freq_band_indices[3], sampling_frequency, nperseg)
    gamma_band_power = compute_band_power(gamma_psd, freq_band_indices[4], sampling_frequency, nperseg)

    spectral_entropy_result = compute_spectral_entropy(np.concatenate([delta_psd, theta_psd, alpha_psd, beta_psd, gamma_psd]))
    # Compute the power spectral density (PSD)
    psd, _ = scipy.signal.welch(signal, fs=sampling_frequency, nperseg=nperseg)

    return [
        delta_band_power,
        theta_band_power,
        alpha_band_power,
        beta_band_power,
        gamma_band_power,
        spectral_entropy_result,
    ]


##Time domain Coeffients

In [ ]:
import numpy as np
from scipy.fft import fft
import pywt
import os
from scipy.stats import entropy, skew, kurtosis
def compute_standard_deviation(data):
    return np.std(data)

def compute_skewness(data):
    return skew(data)

def compute_kurtosis(data):
    return kurtosis(data)

def compute_median(data):
    return np.median(data)

def compute_band_power_time(data, sampling_frequency, nperseg):
    _, power_density = welch(data, fs=sampling_frequency, nperseg=nperseg)
    return np.mean(power_density)
def peak_to_peak_voltage(data):
    return np.ptp(data)

def total_signal_area(data):
    return np.sum(np.abs(data))

def decorrelation_time(data):
    autocorrelation = np.correlate(data, data, mode='full')
    zero_crossings = np.where(np.diff(np.sign(autocorrelation)))[0]

    if len(zero_crossings) > 0:
        first_zero_crossing = zero_crossings[0]
        time_points = np.arange(len(autocorrelation))
        decorrelation_time = time_points[first_zero_crossing]
        return decorrelation_time
    else:
        return -1
def extract_time_domain_features(data,sampling_frequency, nperseg):
    features = [
         compute_standard_deviation(data),
         compute_skewness(data),
       compute_kurtosis(data),
        compute_median(data),
         compute_band_power_time(data, sampling_frequency, nperseg),
         peak_to_peak_voltage(data),
         total_signal_area(data),
        decorrelation_time(data)
    ]
    return features

##Feature Extraction

In [ ]:
import numpy as np
import scipy.signal as signal
from scipy.fft import fft
import pywt
import os
import pandas as pd


def extract_features(segment,feature_names,time):
    num_channels, num_time_points = segment.shape
    df = pd.DataFrame(columns=feature_names)

    for channel_index, channel_data in enumerate(segment):
        extracted_features = []
        wavelet_name = 'db4'
        extracted_features.extend( wavelet_feature_extraction(channel_data, wavelet_name, 256,256))
        extracted_features.extend(extract_time_domain_features(channel_data,256,256))
        extracted_features.extend(extract_frequency_domain_features(channel_data, 256))
        # print('channel',[channel_index])
        # print('time',[time])

        extracted_features.extend([channel_index,time])
        a_series = pd.Series(extracted_features, index=feature_names)
        df = df.append(a_series, ignore_index=True)

    return df

def extract_features_without_segments(data, time_window_duration,feature_names):
    num_channels, num_data_points = data.shape
    data_per_segment = time_window_duration * 256

    num_segments = num_data_points // data_per_segment
    df=pd.DataFrame(columns=feature_names)

    for i in range(num_segments):
        print('segment', i)
        features = extract_features(data[:, i * data_per_segment: (i + 1) * data_per_segment],feature_names,i)
        df = pd.concat([df, features], ignore_index=True)
    return df

In [ ]:
def load_and_process_files(folder_path, time_window_duration,feature_names):
    file_list = [f for f in os.listdir(folder_path) if f.endswith('.npy')]

    df=pd.DataFrame(columns=feature_names)

    for file_name in file_list:
        file_path = os.path.join(folder_path, file_name)

        data = np.load(file_path)
        features = extract_features_without_segments(data, time_window_duration,feature_names)
        df = pd.concat([df, features], ignore_index=True)

        print('file: ',file_name)
        del data

    return df
